In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the pre-trained Inception V3 model
model = models.inception_v3(pretrained=True).to(device)
model.aux_logits = False
# Freeze pre-trained layers
#for param in model.parameters():
#    param.requires_grad = False

# Print layer names and their parameters
#for name, param in model.named_parameters():
#    print(f"Layer: {name}, Parameters: {param.size()}")

'''for name, param in model.named_parameters():
    print(f"Layer: {name}, Parameters: {param.size()}")
    param.requires_grad = False #to freeze
# Replace auxiliary classifier (optional)
if hasattr(model, 'AuxLogits'):
    in_features_aux = model.AuxLogits.fc.in_features
    model.AuxLogits.fc = nn.Linear(in_features_aux, num_classes)
# Modify the final fully connected layer for CIFAR-100
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 100, device=device)  # Change the output size to 100 classes

# Access a specific layer by name
specific_layer = getattr(model, 'layer_name')
# Replace with a new Linear layer
new_linear_layer = nn.Linear(new_in_features, new_out_features)
setattr(model, 'layer_name', new_linear_layer)
# Remove a layer
delattr(model, 'layer_name')
# Add a new layer
new_layer = nn.Linear(new_in_features, new_out_features)
setattr(model, 'new_layer_name', new_layer)
'''

# Modify the final fully connected layer for CIFAR-100
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 100, device=device)  # Change the output size to 100 classes

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:04<00:00, 25.1MB/s]


In [3]:
# Add more layers at the end
class ExtendedInceptionV3(nn.Module):
    def __init__(self, base_model):
        super(ExtendedInceptionV3, self).__init__()
        self.base_model = base_model  # Pre-trained Inception V3
        self.additional_layers = nn.Sequential(
            nn.Linear(100, 50),  # Example: Add a linear layer with 50 output features
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(50, 100)  # Example: Add another linear layer with 10 output features
        )

    def forward(self, x):
        x = self.base_model(x)
        x = self.additional_layers(x)
        return x

In [4]:
# Create an instance of the extended model
extended_model = ExtendedInceptionV3(model).to(device)

# Define data transformations (same as before)
transform = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load CIFAR-100 dataset
train_dataset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

100%|██████████| 169001437/169001437 [00:13<00:00, 12892050.21it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [7]:
# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(extended_model.parameters(), lr=0.001, momentum=0.9)

In [6]:
# Train the extended model
num_epochs = 2
for epoch in range(num_epochs):
    extended_model.train()
    running_loss = 0.0
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = extended_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        # Print training information
        if batch_idx % 100 == 0:
            print(f"Epoch {epoch}/{num_epochs}, Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item():.4f}")

    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}")

Epoch 0/2, Batch 0/782, Loss: 4.5914
Epoch 0/2, Batch 100/782, Loss: 4.6154
Epoch 0/2, Batch 200/782, Loss: 4.6134
Epoch 0/2, Batch 300/782, Loss: 4.5840
Epoch 0/2, Batch 400/782, Loss: 4.5859
Epoch 0/2, Batch 500/782, Loss: 4.5868
Epoch 0/2, Batch 600/782, Loss: 4.5621
Epoch 0/2, Batch 700/782, Loss: 4.5439
Epoch 1/2, Loss: 4.5858
Epoch 1/2, Batch 0/782, Loss: 4.5208
Epoch 1/2, Batch 100/782, Loss: 4.5062
Epoch 1/2, Batch 200/782, Loss: 4.4508
Epoch 1/2, Batch 300/782, Loss: 4.4511
Epoch 1/2, Batch 400/782, Loss: 4.2495
Epoch 1/2, Batch 500/782, Loss: 4.2046
Epoch 1/2, Batch 600/782, Loss: 4.1378
Epoch 1/2, Batch 700/782, Loss: 3.9827
Epoch 2/2, Loss: 4.2451


In [8]:
# Save the trained model (optional)
torch.save(extended_model.state_dict(), 'extended_inception_withoutfreeze.pth')

In [9]:
# Evaluate on the test set
extended_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = extended_model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Accuracy on the test set: {accuracy:.4f}")


Accuracy on the test set: 0.2404


In [10]:
extended_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = extended_model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Accuracy on the train set: {accuracy:.4f}")

Accuracy on the train set: 0.2435
